[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/rag/multimodal_rag_mongodb_voyage_ai.ipynb)

[![View Article](https://img.shields.io/badge/View%20Article-blue)](TBD)

# Building Multimodal RAG Applications with MongoDB and Voyage AI

This notebook evaluates two different techniques for multimodal RAG:

### 1. Using CLIP-based models
* Use a multimodal embedding model to embed images and text chunks
* Retrieve both using vector search
* Pass raw images and text chunks to a multimodal LLM for answer synthesis

### 2. Using VLM-based Voyage AI models
* Take screenshot of full pages
* Retrieve screenshots using semantic search
* Pass the screenshot to a multimodal LLM for answer synthesis

# Step 1: Install required libraries

In [33]:
!pip install -qU pymongo voyageai langchain sentence-transformers PyMuPDF Pillow tqdm


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# Step 2: Setup prerequisites

In [79]:
import getpass
import os

from pymongo import MongoClient

In [ ]:
DB_NAME = "multimodal_rag"
COLLECTION_NAME = "voyage_ai"
VS_INDEX_NAME = "vector_index"
VOYAGE_EMBEDDING_MODEL = "voyage-multimodal-3"
CLIP_EMBEDDING_MODEL = "google/siglip-base-patch16-224"
LLM = "gemini-2.0-flash"
S3_BUCKET = "genai-tutorials"

### MongoDB

In [38]:
# Your MongoDB connection string
MONGODB_URI = getpass.getpass("Enter your MongoDB connection string: ")

Enter your MongoDB connection string:  ········


{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1742333710, 1),
  'signature': {'hash': b'\xbf\xdd\x93\xf1$\x81q=\xe3\xcd\x90l\x14\xcf\x13\xbfX\x9cI\xad',
   'keyId': 7421985632479608834}},
 'operationTime': Timestamp(1742333710, 1)}

In [ ]:
# Create a MongoDB client
mongodb_client = MongoClient(
    MONGODB_URI, appname="devrel.showcase.multimodal_rag_mongodb_voyage_ai"
)
# Check connection to the cluster
mongodb_client.admin.command("ping")

### Voyage AI

In [4]:
# Set Voyage AI API Key
os.environ["VOYAGE_API_KEY"] = getpass.getpass("Enter your Voyage AI API key: ")

Enter your Voyage AI API key:  ········


### Google

In [5]:
# Set Gemini API Key
os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Gemini AI API key: ")

Enter your Gemini AI API key:  ········


### AWS

In [75]:
# Set AWS secrets
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter your AWS access key ID: ")

Enter your AWS Access Key ID:  ········


In [76]:
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass(
    "Enter your AWS secret access key: "
)

Enter your AWS secret access key:  ········


# Step 3: Read PDF from URL

In [6]:
from io import BytesIO

import pymupdf
import requests

In [7]:
response = requests.get("https://arxiv.org/pdf/2501.12948")
if response.status_code != 200:
    raise ValueError(f"Failed to download PDF. Status code: {response.status_code}")

pdf_stream = BytesIO(response.content)
pdf = pymupdf.open(stream=pdf_stream, filetype="pdf")

# Step 4: Convert PDF pages to images and store them in AWS S3

In [78]:
import gzip

import boto3
from PIL import Image
from tqdm import tqdm

In [119]:
s3 = boto3.client("s3")

In [93]:
def upload_image_to_s3(key, data):
    buffer = BytesIO()
    with gzip.GzipFile(fileobj=buffer, mode="wb", compresslevel=9) as f:
        f.write(data)
    try:
        s3.put_object(
            Bucket=S3_BUCKET,
            Key=key,
            Body=buffer.getvalue(),
            ContentEncoding="gzip",
            ContentType="image/png",
        )
    except Exception as e:
        print(e)

In [94]:
def get_embedding(data, input_type):
    embedding = voyageai_client.multimodal_embed(
        inputs=[[data]], model=EMBEDDING_MODEL, input_type=input_type
    ).embeddings[0]
    return embedding

In [122]:
images = []

In [123]:
zoom = 3.0
mat = pymupdf.Matrix(zoom, zoom)
for n in tqdm(range(pdf.page_count)):
    pix = pdf[n].get_pixmap(matrix=mat)
    img_bytes = pix.samples
    img_key = f"multimodal-rag/{n}.png.gz"
    upload_image_to_s3(img_key, img_bytes)
    images.append(img_bytes)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:16<00:00,  1.31it/s]


# Step 5: Embed the images

In [130]:
from sentence_transformers import SentenceTransformer
from voyageai import Client

In [124]:
voyageai_client = Client()

In [132]:
siglip = SentenceTransformer("clip-ViT-B-32")

No sentence-transformers model found with name sentence-transformers/clip-ViT-B-32. Creating a new one with mean pooling.


OSError: sentence-transformers/clip-ViT-B-32 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [105]:
zoom = 3.0
mat = pymupdf.Matrix(zoom, zoom)
for n in tqdm(range(pdf.page_count)):
    temp = {}
    pix = pdf[n].get_pixmap(matrix=mat)
    img_bytes = pix.samples
    img_key = f"multimodal-rag/{n}.png.gz"
    upload_image_to_s3(img_key, img_bytes)
    img_width = pix.width
    img_height = pix.height
    img = Image.frombytes("RGB", [img_width, img_height], img_bytes)
    temp["s3_key"] = img_key
    print("Embedding the image")
    temp["embedding"] = get_embedding(img, "document")
    temp["width"] = img_width
    temp["height"] = img_height
    docs.append(temp)

  0%|                                                                                                                                                                                                                                                                                                                                              | 0/22 [00:00<?, ?it/s]

Uploading image to S3
Embedding the image


  5%|██████████████▊                                                                                                                                                                                                                                                                                                                       | 1/22 [00:02<00:44,  2.11s/it]

Uploading image to S3
Embedding the image


  9%|█████████████████████████████▋                                                                                                                                                                                                                                                                                                        | 2/22 [00:03<00:34,  1.70s/it]

...

 91%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                             | 20/22 [00:31<00:02,  1.41s/it]

Uploading image to S3
Embedding the image


 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏              | 21/22 [00:33<00:01,  1.38s/it]

Uploading image to S3
Embedding the image


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:34<00:00,  1.56s/it]


In [109]:
docs[0]

{'s3_key': 'multimodal-rag/0.png.gz',
 'embedding': [0.005950927734375,
  0.018798828125,
  0.040283203125,
  -0.006500244140625,
  0.00714111328125,
  -0.027099609375,
  -0.0439453125,
  -0.0031585693359375,
  0.018798828125,
  0.0179443359375,
  -0.0262451171875,
  0.041259765625,
  0.00537109375,
  0.00396728515625,
  -0.0035858154296875,
  -0.001251220703125,
  -0.045166015625,
  -0.0177001953125,
  0.021484375,
  -0.040283203125,
  0.03271484375,
  -0.00148773193359375,
  0.046142578125,
  -0.04736328125,
  -0.00897216796875,
  0.043212890625,
  -0.0189208984375,
  -0.044189453125,
  0.01104736328125,
  1.1146068572998047e-05,
  -0.00677490234375,
  -0.01708984375,
  0.030029296875,
  0.01263427734375,
  0.007232666015625,
  -0.029296875,
  -0.023193359375,
  0.01324462890625,
  0.0269775390625,
  -0.01434326171875,
  -0.078125,
  0.0062255859375,
  -0.10107421875,
  -0.0262451171875,
  -0.000942230224609375,
  0.032470703125,
  -0.055908203125,
  0.0137939453125,
  0.010498046875

In [107]:
# Retrieval
response = s3.get_object(Bucket=S3_BUCKET, Key="multimodal-rag/0.png.gz")
compressed_data = response["Body"].read()

# Decompress
buffer = BytesIO(compressed_data)
with gzip.GzipFile(fileobj=buffer, mode="rb") as f:
    image_bytes = f.read()

# Convert to PIL Image
img = Image.frombytes("RGB", [docs[0]["width"], docs[0]["height"]], image_bytes)

# Step 6: Ingest documents into MongoDB

In [110]:
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [111]:
collection.delete_many({})

DeleteResult({'n': 22, 'electionId': ObjectId('7fffffff0000000000000026'), 'opTime': {'ts': Timestamp(1742338503, 22), 't': 38}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1742338503, 22), 'signature': {'hash': b"P\xedA\xd8\xe4\x16p\xf9\x8a'\x1f\xab\xf1.\x82\xf4@\x83V\xb7", 'keyId': 7421985632479608834}}, 'operationTime': Timestamp(1742338503, 22)}, acknowledged=True)

In [112]:
collection.insert_many(docs)

InsertManyResult([ObjectId('67d9f9c75b23b0aee3d618b7'), ObjectId('67d9f9c75b23b0aee3d618b8'), ObjectId('67d9f9c75b23b0aee3d618b9'), ObjectId('67d9f9c75b23b0aee3d618ba'), ObjectId('67d9f9c75b23b0aee3d618bb'), ObjectId('67d9f9c75b23b0aee3d618bc'), ObjectId('67d9f9c75b23b0aee3d618bd'), ObjectId('67d9f9c75b23b0aee3d618be'), ObjectId('67d9f9c75b23b0aee3d618bf'), ObjectId('67d9f9c75b23b0aee3d618c0'), ObjectId('67d9f9c75b23b0aee3d618c1'), ObjectId('67d9f9c75b23b0aee3d618c2'), ObjectId('67d9f9c75b23b0aee3d618c3'), ObjectId('67d9f9c75b23b0aee3d618c4'), ObjectId('67d9f9c75b23b0aee3d618c5'), ObjectId('67d9f9c75b23b0aee3d618c6'), ObjectId('67d9f9c75b23b0aee3d618c7'), ObjectId('67d9f9c75b23b0aee3d618c8'), ObjectId('67d9f9c75b23b0aee3d618c9'), ObjectId('67d9f9c75b23b0aee3d618ca'), ObjectId('67d9f9c75b23b0aee3d618cb'), ObjectId('67d9f9c75b23b0aee3d618cc')], acknowledged=True)

# Step 7: Create a vector search index

In [73]:
model = {
    "name": VS_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1024,
                "similarity": "cosine",
            }
        ]
    },
}

In [74]:
collection.create_search_index(model=model)

'vector_index'

# Step 8: Retrieve documents using vector search

In [115]:
def vector_search(user_query):
    query_embedding = get_embedding(user_query, "query")
    pipeline = [
        {
            "$vectorSearch": {
                "index": VS_INDEX_NAME,
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,
                "limit": 5,
            }
        },
        {
            "$project": {
                "_id": 0,
                "s3_key": 1,
                "width": 1,
                "height": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        },
    ]

    # Execute the aggregation `pipeline` and store the results in `results`
    results = collection.aggregate(pipeline)
    return list(results)

In [116]:
vector_search("Summarize the benchmark performance of Deepseek R1.")

[{'s3_key': 'multimodal-rag/0.png.gz',
  'width': 1786,
  'height': 2526,
  'score': 0.7545373439788818},
 {'s3_key': 'multimodal-rag/12.png.gz',
  'width': 1786,
  'height': 2526,
  'score': 0.7498551607131958},
 {'s3_key': 'multimodal-rag/13.png.gz',
  'width': 1786,
  'height': 2526,
  'score': 0.7284250259399414},
 {'s3_key': 'multimodal-rag/6.png.gz',
  'width': 1786,
  'height': 2526,
  'score': 0.7090297341346741},
 {'s3_key': 'multimodal-rag/3.png.gz',
  'width': 1786,
  'height': 2526,
  'score': 0.7054796814918518}]